In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder 
from sklearn.preprocessing import MinMaxScaler

### Reading dataset

In [2]:
dataset1 = pd.read_csv('datasets/1/WA_Fn-UseC_-Telco-Customer-Churn.csv')
dataset2 = pd.read_csv('datasets/2/adult.data')
dataset3 = pd.read_csv('datasets/3/creditcard.csv')

dataset = dataset1

In [3]:
dataset.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [189]:
dataset.drop(['customerID'], axis=1, inplace=True)
dataset.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [190]:
X = dataset.iloc[:, :-1].values
print(X)

[['Female' 0 'Yes' ... 'Electronic check' 29.85 '29.85']
 ['Male' 0 'No' ... 'Mailed check' 56.95 '1889.5']
 ['Male' 0 'No' ... 'Mailed check' 53.85 '108.15']
 ...
 ['Female' 0 'Yes' ... 'Electronic check' 29.6 '346.45']
 ['Male' 1 'Yes' ... 'Mailed check' 74.4 '306.6']
 ['Male' 0 'No' ... 'Bank transfer (automatic)' 105.65 '6844.5']]


In [193]:
Y = dataset.iloc[:, -1].values
# label encode y
labelencoder_Y = LabelEncoder()
Y = labelencoder_Y.fit_transform(Y)
print(Y)

[0 0 1 ... 0 1 0]


In [192]:
X_df = pd.DataFrame(X)
X_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.3,1840.75
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.7,151.65


In [ ]:
def global_preprocess(X, 
                      columns_to_hot_encode, 
                      columns_to_label_encode,
                      columns_to_normalize,
                      fill_missing_type='mean'):

    
    #one hot encode
    one_hot_encoder = OneHotEncoder(sparse=False, drop='first')
    X_df_encoded = one_hot_encoder.fit_transform(X_df[columns_to_hot_encode])
    X_df = X_df.drop(columns_to_hot_encode, axis=1)
    X_df = pd.concat([X_df, pd.DataFrame(X_df_encoded)], axis=1)
    
    # label encode
    label_encoder = LabelEncoder()
    for column in columns_to_label_encode:
        X_df[column] = label_encoder.fit_transform(X_df[column])
        
    #convert to numeric and fill missing values
    X_df = X_df.apply(pd.to_numeric, errors='coerce')
    if fill_missing_type == 'zero':
        X_df = X_df.fillna(0)
    else:    
        X_df = X_df.fillna(X_df.mean())
            
    #normalize
    scaler = MinMaxScaler()
    X_df[columns_to_normalize] = scaler.fit_transform(X_df[columns_to_normalize])
    
    
    return X_df
    

In [5]:
def preprocess_dataset1(X):
    preprocessed_X = pd.DataFrame(X)
    #preprocessed_X = preprocessed_X.drop(preprocessed_X.columns[0], axis=1).reset_index(drop=True)

    preprocessed_X[6] = preprocessed_X[6].replace('No phone service', 'No')
    for i in range(8, 14):
        preprocessed_X[i] = preprocessed_X[i].replace('No internet service', 'No')


    # one hot encoding
    one_hot_encoder = OneHotEncoder(sparse=False, drop='first')  
    columns_to_hot_encode = [7, 14, 16]
    one_hot_encoded = one_hot_encoder.fit_transform(preprocessed_X[columns_to_hot_encode])
    one_hot_column_names = [f'one_hot_{i}' for i in range(one_hot_encoded.shape[1])]
    preprocessed_X = preprocessed_X.drop(columns_to_hot_encode, axis=1)
    preprocessed_X = pd.concat([preprocessed_X, pd.DataFrame(one_hot_encoded, columns=one_hot_column_names).astype(int)], axis=1)
    preprocessed_X = preprocessed_X.reset_index(drop=True)
    
    # Label encoding binary categorical features
    columns_to_label_encode = [0, 1, 2, 3, 5, 6, 8, 9, 10, 11, 12, 13, 15]  
    label_encoder = LabelEncoder()
    for column in columns_to_label_encode:
        preprocessed_X[column] = label_encoder.fit_transform(preprocessed_X[column])
        

    # convert to numeric and fill missing values
    preprocessed_X = preprocessed_X.apply(pd.to_numeric, errors='coerce')
    preprocessed_X = preprocessed_X.fillna(0)
    
    
    # apply normalization to specific columns
    scaler = MinMaxScaler()
    columns_to_normalize = [4, 17, 18]
    preprocessed_X[columns_to_normalize] = scaler.fit_transform(preprocessed_X[columns_to_normalize])

    
    return preprocessed_X

In [221]:
preprocessed_X = preprocess_dataset1(X)
# adjust columns so that it starts from 0
#preprocessed_X.columns = range(preprocessed_X.shape[1])
preprocessed_X = preprocessed_X.reset_index(drop=True)

preprocessed_X.head()

c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,0,1,2,3,4,5,6,8,9,10,...,15,17,18,one_hot_0,one_hot_1,one_hot_2,one_hot_3,one_hot_4,one_hot_5,one_hot_6
0,0,0,1,0,0.013889,0,0,0,1,0,...,1,0.115423,0.003437,0,0,0,0,0,1,0
1,1,0,0,0,0.472222,1,0,1,0,1,...,0,0.385075,0.217564,0,0,1,0,0,0,1
2,1,0,0,0,0.027778,1,0,1,1,0,...,1,0.354229,0.012453,0,0,0,0,0,0,1
3,1,0,0,0,0.625000,0,0,1,0,1,...,0,0.239303,0.211951,0,0,1,0,0,0,0
4,0,0,0,0,0.027778,1,0,0,0,0,...,1,0.521891,0.017462,1,0,0,0,0,1,0


In [222]:
# Assuming df is your DataFrame
for column in preprocessed_X.columns:
    num_unique_values = preprocessed_X[column].nunique()
    print(f"column '{column}' is of type '{preprocessed_X[column].dtype}'")

column '0' is of type 'int32'
column '1' is of type 'int32'
column '2' is of type 'int32'
column '3' is of type 'int32'
column '4' is of type 'float64'
column '5' is of type 'int32'
column '6' is of type 'int32'
column '8' is of type 'int32'
column '9' is of type 'int32'
column '10' is of type 'int32'
column '11' is of type 'int32'
column '12' is of type 'int32'
column '13' is of type 'int32'
column '15' is of type 'int32'
column '17' is of type 'float64'
column '18' is of type 'float64'
column 'one_hot_0' is of type 'int32'
column 'one_hot_1' is of type 'int32'
column 'one_hot_2' is of type 'int32'
column 'one_hot_3' is of type 'int32'
column 'one_hot_4' is of type 'int32'
column 'one_hot_5' is of type 'int32'
column 'one_hot_6' is of type 'int32'


In [8]:
def entropy(labels):
    unique_labels, counts = np.unique(labels, return_counts=True)
    probabilities = counts / len(labels)
    entropy_value = -np.sum(probabilities * np.log2(probabilities))
    return entropy_value

In [9]:
def feature_selection_by_IG(preprocessed_X, Y, num_features=-1):
    base_entropy = entropy(Y)
    print(f"base entropy: {base_entropy}")
    
    
    # calculate entropy for each feature
    entropies = []
    InfoGain = []
    for column in preprocessed_X.columns:
        feat_value, feat_value_counts = np.unique(preprocessed_X[column], return_counts=True)
        weighted_feature_entropy = 0
        
        for value, count in zip(feat_value, feat_value_counts):
            subset_Y = Y[preprocessed_X[column] == value]
            weighted_feature_entropy += count / len(preprocessed_X[column]) * entropy(subset_Y)
            
        entropies.append(weighted_feature_entropy)
        InfoGain.append(base_entropy - weighted_feature_entropy)
        print(f"entropy for column '{column}': {weighted_feature_entropy} and information gain: {base_entropy - weighted_feature_entropy}" )
        
        # probabilities = value_counts / len(preprocessed_X[column])
        # entropy = -np.sum(probabilities * np.log2(probabilities))
        # entropies.append(entropy)
        # InfoGain.append(base_entropy - entropy)
        # print(f"entropy for column '{column}': {entropy} and information gain: {base_entropy - entropy}" )
    
    # sort by InfoGain
    sorted_indices = np.argsort(InfoGain)[::-1]
    sorted_IG = np.sort(InfoGain)[::-1]
    sorted_columns = preprocessed_X.columns[sorted_indices]
    
    print(sorted_IG)
    
    if num_features == -1:
        num_features = len(sorted_columns)
        
    # return dataframe with selected features
    return pd.DataFrame(preprocessed_X[sorted_columns[:num_features]], columns=sorted_columns[:num_features])

        
    

In [118]:
# def feature_selection_by_IG(features, target, num_features=-1):
#     _, label_counts = np.unique(target, return_counts=True)
#     probs = label_counts / len(target)
#     base_entropy = np.sum(-(probs * np.log2(probs)))
#     #print(f"base entropy: {base_entropy}")
    
#     feature_IGs = []
#     for i in features:
#         _feats, label_counts_feats = np.unique(i, return_counts=True)
#         probs_feats = label_counts_feats / len(i)
#         entropy_feats = np.sum(-(probs_feats * np.log2(probs_feats)))
#         information_gain = base_entropy - entropy_feats
#         feature_IGs.append(information_gain)
        
#     # calculate      
#     feature_IGs = np.array(feature_IGs)
#     sorted_index = np.argsort(feature_IGs)[::-1]
#     sorted_features = features[sorted_index]
#     sorted_IGs = feature_IGs[sorted_index]
    
#     if num_features == -1:
#         num_features = len(features)
#     # return array of features and array of IGs
#     return sorted_features[:num_features], sorted_IGs[:num_features]
    
    

In [316]:
feature_selection_by_IG(preprocessed_X, Y, 5)

base entropy: 0.8347419473972113
entropy for column '0': 0.8346884480604979 and information gain: 5.34993367133918e-05
entropy for column '1': 0.8194821811446197 and information gain: 0.015259766252591622
entropy for column '2': 0.8182178128778077 and information gain: 0.0165241345194036
entropy for column '3': 0.8138701014961185 and information gain: 0.02087184590109281
entropy for column '4': 0.720323151904608 and information gain: 0.11441879549260325
entropy for column '5': 0.8346378432561371 and information gain: 0.00010410414107420163
entropy for column '6': 0.8335863937829578 and information gain: 0.0011555536142534573
entropy for column '8': 0.8118251356561681 and information gain: 0.022916811741043208
entropy for column '9': 0.8297502033535026 and information gain: 0.0049917440437087235
entropy for column '10': 0.8315283062017049 and information gain: 0.003213641195506378
entropy for column '11': 0.8136664332127168 and information gain: 0.021075514184494493
entropy for column '

,18,17,4,one_hot_3,one_hot_0
0,0.003437,0.115423,0.013889,0,0
1,0.217564,0.385075,0.472222,0,0
2,0.012453,0.354229,0.027778,0,0
3,0.211951,0.239303,0.625000,0,0
4,0.017462,0.521891,0.027778,0,1
...,...,...,...,...,...
7038,0.229194,0.662189,0.333333,0,0
7039,0.847792,0.845274,1.000000,0,1
7040,0.039892,0.112935,0.152778,0,0
7041,0.035303,0.558706,0.055556,0,1


In [317]:
def split_dataset(X, Y, train_ratio=0.8):
    train_size = int(len(X) * train_ratio)
    X_train, X_test = X[:train_size], X[train_size:]
    Y_train, Y_test = Y[:train_size], Y[train_size:]
    return X_train, X_test, Y_train, Y_test

In [318]:
X_train, X_test, Y_train, Y_test = split_dataset(preprocessed_X, Y, train_ratio=0.8)

In [319]:
# print test train heads and dimentions
print(X_train.head())
#print(X_test.head())
#print(Y_train)
#print(Y_test)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

   0  1  2  3         4  5  6  8  9  10  ...  15        17        18  \
0  0  0  1  0  0.013889  0  0  0  1   0  ...   1  0.115423  0.003437   
1  1  0  0  0  0.472222  1  0  1  0   1  ...   0  0.385075  0.217564   
2  1  0  0  0  0.027778  1  0  1  1   0  ...   1  0.354229  0.012453   
3  1  0  0  0  0.625000  0  0  1  0   1  ...   0  0.239303  0.211951   
4  0  0  0  0  0.027778  1  0  0  0   0  ...   1  0.521891  0.017462   

   one_hot_0  one_hot_1  one_hot_2  one_hot_3  one_hot_4  one_hot_5  one_hot_6  
0          0          0          0          0          0          1          0  
1          0          0          1          0          0          0          1  
2          0          0          0          0          0          0          1  
3          0          0          1          0          0          0          0  
4          1          0          0          0          0          1          0  

[5 rows x 23 columns]
(5634, 23)
(1409, 23)
(5634,)
(1409,)


In [321]:
class modified_regressor:
    def __init__(self, X, Y, data_point_weights, threshold, max_feature_count=-1, learning_rate=0.1):
        self.X = X
        self.Y = Y
        self.y_hat = None
        self.data_point_weights = data_point_weights
        self.weights = None
        self.max_feature_count = max_feature_count
        self.threshold = threshold
        self.learning_rate = learning_rate
        self.selected_features = None
        
        self.sorted_indices = None
        self.sorted_IG = None
        self.sorted_columns = None
        

    def feature_selection_by_IG(self):
        preprocessed_X = self.X
        Y = self.Y
        _, label_counts = np.unique(Y, return_counts=True)
        probabilties = label_counts / len(Y)
        base_entropy = -np.sum(probabilties * np.log2(probabilties))
        print(f"base entropy: {base_entropy}")
        
        
        # calculate entropy for each feature
        entropies = []
        InfoGain = []
        preprocessed_X = pd.DataFrame(preprocessed_X)
        for column in preprocessed_X.columns:
            _, value_counts = np.unique(preprocessed_X[column], return_counts=True)
            probabilities = value_counts / len(preprocessed_X[column])
            entropy = -np.sum(probabilities * np.log2(probabilities))
            entropies.append(entropy)
            InfoGain.append(base_entropy - entropy)
            #print(f"entropy for column '{column}': {entropy} and information gain: {base_entropy - entropy}" )
        
        # sort by InfoGain
        self.sorted_indices = np.argsort(InfoGain)
        self.sorted_IG = np.sort(InfoGain)
        self.sorted_columns = preprocessed_X.columns[self.sorted_indices]
        
        #print(sorted_IG)
        
        if self.max_feature_count == -1:
            self.max_feature_count = len(self.sorted_columns)
            
        # return dataframe with selected features
        self.selected_features = self.sorted_columns[:self.max_feature_count]
        self.X = pd.DataFrame(preprocessed_X[self.sorted_columns[:self.max_feature_count]], columns=self.sorted_columns[:self.max_feature_count])
    
    #calculating approximate integer ratio of the data_point_weights
    def calculate_integer_ratio(self):
        integer_ratio = []
        for i in range(len(self.data_point_weights)):
            integer_ratio.append(round(self.data_point_weights[i] / min(self.data_point_weights)))
        return integer_ratio

    # duplicate the data points according to their weights
    def duplicate_data_points(self):
        integer_ratio = self.calculate_integer_ratio()
        new_X = []
        new_Y = []
        
        #X_columns = self.X.columns

        for i in range(len(integer_ratio)):
            for j in range(integer_ratio[i]):
                # append duplicate tuple
                new_X.append(self.X.iloc[i])    
                new_Y.append(self.Y[i])
                

        self.X = np.array(new_X)
        self.Y = np.array(new_Y)

    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def binary_cross_entropy(self, y, y_hat):
        return -(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))
    
    def gradient_descent(self, X, y):
        # calculate gradient
        z = np.dot(X, self.weights)
        y_pred = self.sigmoid(z)
        gradient = np.dot(X.T, (y_pred - y)) / y.size
        # update weights
        self.weights -= self.learning_rate * gradient
    
    def check_stuffs(self):
        print("hello")
    
    def train(self):
        # # update X, Y to duplicated X, Y
        # self.duplicate_data_points()
        
        # # feature selection
        # self.feature_selection_by_IG()
        
        self.weights = np.zeros(self.X.shape[1])
        
        if self.threshold is not None and self.threshold > 0:
            error = float('inf')
            iteration = 0
            
            while error > self.threshold:
                self.gradient_descent(self.X, self.Y)
                iteration += 1
                z = np.dot(self.X, self.weights)
                y_hat = self.sigmoid(z)
                sum_error = np.sum(self.binary_cross_entropy(self.Y, y_hat))
                error = sum_error / len(self.Y)
        else:
            steps = 5000
            for i in range(steps):
                self.gradient_descent(self.X, self.Y)
                z = np.dot(self.X, self.weights)
                y_hat = self.sigmoid(z)
                sum_error = np.sum(self.binary_cross_entropy(self.Y, y_hat))
                error = sum_error / len(self.Y)
        
        print(f"total iterations: {iteration}")        
        #return self.weights, error, iteration
        
    def predict(self, X_test):
        # feature select from X_test
        #pd.DataFrame(preprocessed_X[sorted_columns[:num_features]], columns=sorted_columns[:num_features])
        
        #X_test = pd.DataFrame(X_test[self.sorted_columns[:self.max_feature_count]], columns=self.sorted_columns[:self.max_feature_count])
        #print(f"shape of X_test: {X_test.shape}")
        
        
        z = np.dot(X_test, self.weights)
        y_hat = np.round(self.sigmoid(z))
        self.y_hat = y_hat
        return y_hat
    
    def accuracy(self, y_test):
        return (np.sum(self.y_hat == y_test) / len(y_test))*100
                

In [329]:
# initialize data point weights with 1/N
data_point_weights = np.ones(len(X_train)) / len(X_train)

# initialize regressor
regressor = modified_regressor(X_train, Y_train, data_point_weights, 0.42, -1, 0.07)
# train regressor

# feature selection
#regressor.feature_selection_by_IG()


#X_train.head()
# update X, Y to duplicated X, Y
#regressor.duplicate_data_points()

print(f"shape of X: {regressor.X.shape}")
regressor.train()


# predict
regressor.predict(X_test)
#calculate accuracy
accuracy = regressor.accuracy(Y_test)
print(f"accuracy: {accuracy}")


shape of X: (5634, 23)
total iterations: 784
accuracy: 79.55997161107167


In [330]:
def adaboost(examples_X, examples_Y, L_weak, K, num_features=-1):
    """
    Parameters
    ----------
    
    examples : set of N examples
    L_weak : weak learner (logistic regression)
    K : number of weak learners to use
    
    ------------------
    """
    
    data_point_weights = np.ones(len(examples_X)) / len(examples_X)
    hypothesises = []
    hypo_weights = []
    
    #feature selection
    examples_X, examples_Y = feature_selection_by_IG(examples_X, examples_Y, num_features)
    
    for i in range(K):
        #resampling data
        indices = np.random.choice(len(examples_X), len(examples_X), p=data_point_weights)
        resampled_examples_X = examples_X[indices]
        resampled_examples_Y = examples_Y[indices]
        
        # initialize regressor
        regressor = modified_regressor(resampled_examples_X, resampled_examples_Y, data_point_weights, 0.5, -1)
        #hypothesises.append(regressor)
        
        error = 0
        for j in range(len(examples_X)):
            if regressor.predict(examples_X[j]) != examples_Y[j]:
                error += data_point_weights[j]
            
        if error > 0.5:
            continue
        else:
            hypothesises.append(regressor)
        
        for j in range(len(examples_X)):
            if regressor.predict(examples_X[j]) == examples_Y[j]:
                data_point_weights[j] *= error / (1 - error)
        
        # normalizing weights
        data_point_weights /= np.sum(data_point_weights)
        
        hypo_weights.append(np.log2((1 - error) / error))
        
    return hypothesises, hypo_weights

In [331]:
def weighted_majority(hypothesises, hypo_weights, examples_X):
    predictions = []
    for i in range(len(examples_X)):
        prediction = 0
        for j in range(len(hypothesises)):
            prediction += hypo_weights[j] * hypothesises[j].predict(examples_X[i])
        predictions.append(np.sign(prediction))
    return predictions

In [332]:
    """
    asdf
    function adaboost(examples, L_weak, K) returns a weighted majority hypothesis
    inputs: examples, set of N labeled examples (x1, y1) ... (xN, yN)
            L_weak a learning algorithm
            K, the number of hypothesises in the ensemble
            
    local variables: w, a vector of N example weights, initailly 1/N
    h, a vector of K hypothesises
    z, a vector of K hypothesis weights

    for k = 1 to K do
        data <- Resample(examples, w)
        h[k] <- L_weak(data)
        error <- 0
        for i = 1 to N do
            if h[k](xi) != yi then
                error <- error + wi
        if error > 0.5 then continue
        for i = 1 to N do
            if h[k](xi) == yi then
                wi <- wi * error / (1 - error)
        Normalize(w)
        z[k] <- ln((1 - error) / error)
    return Weighted_majority(h, z)
    """

'\nasdf\nfunction adaboost(examples, L_weak, K) returns a weighted majority hypothesis\ninputs: examples, set of N labeled examples (x1, y1) ... (xN, yN)\n        L_weak a learning algorithm\n        K, the number of hypothesises in the ensemble\n        \nlocal variables: w, a vector of N example weights, initailly 1/N\nh, a vector of K hypothesises\nz, a vector of K hypothesis weights\n\nfor k = 1 to K do\n    data <- Resample(examples, w)\n    h[k] <- L_weak(data)\n    error <- 0\n    for i = 1 to N do\n        if h[k](xi) != yi then\n            error <- error + wi\n    if error > 0.5 then continue\n    for i = 1 to N do\n        if h[k](xi) == yi then\n            wi <- wi * error / (1 - error)\n    Normalize(w)\n    z[k] <- ln((1 - error) / error)\nreturn Weighted_majority(h, z)\n'